# 1. Import Dependencies

In [31]:
!pip install stable-baselines3[extra]
import gym 
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# 2. Load Environment

In [32]:
environment_name = "CartPole-v0"

In [33]:
env = gym.make(environment_name)

In [34]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:22.0
Episode:2 Score:19.0
Episode:3 Score:67.0
Episode:4 Score:13.0
Episode:5 Score:23.0


# 3. Understanding The Environment


In [35]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()


0

In [36]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-4.6417546e+00, -3.0391785e+38,  3.5316849e-01, -7.8366312e+37],
      dtype=float32)

# 3. Train an RL Model

In [48]:
log_path = os.path.join('Training','Logs')

In [49]:
log_path

'Training/Logs'

In [50]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cuda device


In [51]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 582  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 439         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008126304 |
|    clip_fraction        | 0.0977      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00115    |
|    learning_rate        | 0.0003      |
|    loss                 | 9.07        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0167     |
|    value_loss           | 68.9        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [52]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [53]:
model.save(PPO_path)

In [43]:
del model

In [44]:
model = PPO.load('PPO_model', env=env)

FileNotFoundError: [Errno 2] No such file or directory: 'PPO_model.zip'

# 5. Evaluation


In [54]:
evaluate_policy(model, env, n_eval_episodes=10,)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [55]:
env.close()

# 6. Test Model

In [56]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.7734804 , -1.4470999 , -0.00997709,  0.9241185 ], dtype=float32)}]


# 7. Viewing Logs in Tensorboard

In [57]:
training_log_path = os.path.join(log_path, 'PPO_3')

# 8. Adding a callback to the training Stage

In [58]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)
model = PPO('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 579  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 440         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009763412 |
|    clip_fraction        | 0.105       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0065     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.12        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0149     |
|    value_loss           | 50.4        |
-----------------------------------------
----------------

# 9. Changing Policies


In [59]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 528  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 386         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013660354 |
|    clip_fraction        | 0.152       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.683      |
|    explained_variance   | -0.0024     |
|    learning_rate        | 0.0003      |
|    loss                 | 4.22        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0176     |
|    value_loss           | 23          |
-----------------------------------------
----------------

# 10. Using an Alternate Algorithm

In [62]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')
model.save(dqn_path)
model = DQN.load(dqn_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.979    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7233     |
|    time_elapsed     | 0        |
|    total_timesteps  | 44       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.93     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 7479     |
|    time_elapsed     | 0        |
|    total_timesteps  | 148      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.887    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 7409     |
|    time_elapsed     | 0        |
|    total_timesteps  | 238      |
----------------------------------
----------------------------------
| 

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 8529     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2331     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 8537     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2420     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 8553     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2508     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 8797     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4991     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 8812     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5113     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 8802     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5168     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 8812     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7328     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 8816     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7427     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 8828     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7571     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 8831     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9739     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 8842     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9891     |
----------------------------------
Eval num_timesteps=10000, episode_reward=9.60 +/- 0.49
Episode length: 9.60 +/- 0.49
----------------------------------
| eval/               |          |
|    mean_ep_length   | 9.6      |
|    mean_reward      | 9.6      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 10000    |
-----

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 8604     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12052    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 8603     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12125    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 8611     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12247    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 8660     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14593    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 8658     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14661    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 8658     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14733    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 8680     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16900    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 8681     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16990    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 8680     |
|    time_elapsed     | 1        |
|    total_timesteps  | 17059    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 8700     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19270    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 8698     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19326    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 8699     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19420    |
----------------------------------
----------------------------------
| rollout/          